In [2]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.16.1-cp312-cp312-win_amd64.whl.metadata (3.5 kB)
  Using cached tensorflow_intel-2.16.1-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
Using cached tensorflow-2.16.1-cp312-cp312-win_amd64.whl (2.1 kB)
Using cached tensorflow_intel-2.16.1-cp312-cp312-win_amd64.whl (377.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow_hub

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow_hub-0.16.1-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached tf_keras-2.16.0-py3-none-any.whl.metadata (1.6 kB)
Using cached tensorflow_hub-0.16.1-py2.py3-none-any.whl (30 kB)
Using cached tf_keras-2.16.0-py3-none-any.whl (1.7 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install librosa

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:


import tensorflow as tf
import tensorflow_hub as hub
import librosa
import numpy as np
import pandas as pd


In [11]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import librosa
import numpy as np
import pandas as pd

# Verify if the class map file exists
class_map_path = r'C:\Yamnet\yamnet_class_map.csv'
if not os.path.exists(class_map_path):
    print(f"Class map file not found at {class_map_path}")
else:
    print(f"Class map file found at {class_map_path}")

    # Load the YAMNet model from TensorFlow Hub
    yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
    yamnet_model = hub.load(yamnet_model_handle)

    # Load your audio file
    audio_file_path = r'C:\Yamnet\filtered_audio_2328.wav'

    # Load the audio file and resample to 16 kHz
    waveform, sample_rate = librosa.load(audio_file_path, sr=16000)

    # Ensure


Class map file found at C:\Yamnet\yamnet_class_map.csv


In [12]:
import tensorflow as tf
import tensorflow_hub as hub
import librosa
import numpy as np
import pandas as pd

# Load the YAMNet model from TensorFlow Hub
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)

# Load your audio file
audio_file_path = r'C:\Yamnet\filtered_audio_2328.wav'

# Load the audio file and resample to 16 kHz
waveform, sample_rate = librosa.load(audio_file_path, sr=16000)

# Ensure the waveform is a 1D array
waveform = waveform.astype(np.float32)

# Run the model - YAMNet expects a 1D tensor
scores, embeddings, spectrogram = yamnet_model(waveform)

# Load the class map for YAMNet from the local path
class_map_path = r'C:\Yamnet\yamnet_class_map.csv'
class_names = list(pd.read_csv(class_map_path)['display_name'])

# Frame duration in seconds (each frame is approximately 0.96 seconds of audio)
frame_duration = 0.96

# Create an empty list to store the segment data
segment_data = []

# Process the scores and map them to class names
for i, score in enumerate(scores):
    top_class_index = np.argmax(score)
    top_class_name = class_names[top_class_index]
    top_class_score = score[top_class_index]
    
    segment_data.append({
        'start_time': i * frame_duration,
        'end_time': (i + 1) * frame_duration,
        'class': top_class_name,
        'score': top_class_score
    })

# Print the segment data
for segment in segment_data:
    print(f"Start: {segment['start_time']}s, End: {segment['end_time']}s, Class: {segment['class']}, Score: {segment['score']}")


Start: 0.0s, End: 0.96s, Class: Boat, Water vehicle, Score: 0.35477933287620544
Start: 0.96s, End: 1.92s, Class: Boat, Water vehicle, Score: 0.32618197798728943
Start: 1.92s, End: 2.88s, Class: Wind noise (microphone), Score: 0.33662477135658264
Start: 2.88s, End: 3.84s, Class: Vehicle, Score: 0.27127736806869507
Start: 3.84s, End: 4.8s, Class: Wind, Score: 0.17338469624519348
Start: 4.8s, End: 5.76s, Class: Eruption, Score: 0.2221900224685669
Start: 5.76s, End: 6.72s, Class: Wind noise (microphone), Score: 0.3510485589504242
Start: 6.72s, End: 7.68s, Class: Wind noise (microphone), Score: 0.2301047444343567


In [14]:

# Iterate over the scores to get the top 3 predictions for each segment
# Looping through each frame to extract the top 3 predictions and their probabilities
for j in range(scores.shape[0]):
    # Get the start and end time of the segment
    # Calculating the start and end time for each audio segment
    start_time = j * frame_duration
    end_time = start_time + frame_duration

    # Get the top 3 predictions and their probabilities
    # Extracting the indices of the top 3 predictions
    top_indices = np.argsort(scores[j])[-3:][::-1]
    # Getting the class names for the top 3 predictions
    top_predictions = [class_names[idx] for idx in top_indices]
    # Getting the probabilities for the top 3 predictions
    top_probabilities = [scores[j, idx] for idx in top_indices]

    # Append the data to the segment_data list
    # Storing the segment times, predictions, and probabilities in a dictionary
    segment_data.append({
        'Segment Start Time': start_time,
        'Segment End Time': end_time,
        '1st Prediction': top_predictions[0],
        '1st Probability': top_probabilities[0],
        '2nd Prediction': top_predictions[1],
        '2nd Probability': top_probabilities[1],
        '3rd Prediction': top_predictions[2],
        '3rd Probability': top_probabilities[2],
    })

# Create a DataFrame from the segment_data list
# Converting the list of dictionaries into a pandas DataFrame for easy manipulation and export
df = pd.DataFrame(segment_data)

# Save the DataFrame to a CSV file
# Exporting the DataFrame to a CSV file for later use or analysis
output_file_path = 'yamnet_predictions.csv'
df.to_csv(output_file_path, index=False)

# Inform the user that the DataFrame has been saved
print(f"DataFrame saved to {output_file_path}")

DataFrame saved to yamnet_predictions.csv
